import packages we need...

In [1]:
import json
from fhirclient import client
import fhirclient.models.specimen as s
import fhirclient.models.patient as p
import fhirclient.models.fhirreference as fr
import fhirclient.models.codeableconcept as cc
import fhirclient.models.coding as co

Create a Specimen object and take a look at its `.elementProperties()`.

In [2]:
mySpecimen = s.Specimen()
for eproperty in mySpecimen.elementProperties():
    print(eproperty)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('accessionIdentifier', 'accessionIdentifier', <class 'fhirclient.models.identifier.Identifier'>, False, None, False)
('collection', 'collection', <class 'fhirclient.models.specimen.SpecimenCollection'>, False, None, False)
('container', 'container', <class 'fhirclient.models.specimen.SpecimenContainer'>, True, None, False)
('identifier', 'identifier', <class

Which, if any, are required?

In [3]:
for eproperty in mySpecimen.elementProperties():
    if eproperty[5] == True:
        print(eproperty)

('subject', 'subject', <class 'fhirclient.models.fhirreference.FHIRReference'>, False, None, True)


We'll use the id of the Patient that was posted to the same server.
We see from `.elementProperties()` that myspecimen.subject is a type of `fhirclient.models.fhirreference.FHIRReference`, so we'll build this.

In [4]:
mySubject = fr.FHIRReference()
for eproperty in mySubject.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('display', 'display', <class 'str'>, False, None, False)
('identifier', 'identifier', <class 'fhirclient.models.identifier.Identifier'>, False, None, False)
('reference', 'reference', <class 'str'>, False, None, False)


None of these are required, so we'll use the `reference` and point to the Patient we created.

In [5]:
mySubject.reference = 'Patient/2609'
mySpecimen.subject = mySubject
import json
print(json.dumps(mySpecimen.as_json(), indent=4))

{
    "subject": {
        "reference": "Patient/2609"
    },
    "resourceType": "Specimen"
}


Now we'll add some collection information to this Specimen. 

In [6]:
myCollection = s.SpecimenCollection()
for eproperty in myCollection.elementProperties():    
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('bodySite', 'bodySite', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('collectedDateTime', 'collectedDateTime', <class 'fhirclient.models.fhirdate.FHIRDate'>, False, 'collected', False)
('collectedPeriod', 'collectedPeriod', <class 'fhirclient.models.period.Period'>, False, 'collected', False)
('collector', 'collector', <class 'fhirclient.models.fhirreference.FHIRReference'>, False, None, False)
('method', 'method', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('quantity', 'quantity', <class 'fhirclient.models.quantity.Quantity'>, False, None, False)


None of these are required, but we'll add `bodySite` and `method`. Both are types of `fhirclient.models.codeableconcept.CodeableConcept`. We'll start with `bodySite`.

In [7]:
myBodySite = cc.CodeableConcept()
for eproperty in myBodySite.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('coding', 'coding', <class 'fhirclient.models.coding.Coding'>, True, None, False)
('text', 'text', <class 'str'>, False, None, False)


We'll us `coding` which is a type of `fhirclient.models.coding.Coding`, which is a `is_list` so it must be put into a list.

In [8]:
myBodySiteCoding = co.Coding()
for eproperty in myBodySiteCoding.elementProperties():
    print(eproperty)

('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('id', 'id', <class 'str'>, False, None, False)
('code', 'code', <class 'str'>, False, None, False)
('display', 'display', <class 'str'>, False, None, False)
('system', 'system', <class 'str'>, False, None, False)
('userSelected', 'userSelected', <class 'bool'>, False, None, False)
('version', 'version', <class 'str'>, False, None, False)


Nothing is required, but we'll add `system`, `code`, and `display`

In [9]:
myBodySiteCoding.system = 'http://snomed.info/sct'
myBodySiteCoding.code = '261063000'
myBodySiteCoding.display = 'Buccal space'
myBodySite.coding = [myBodySiteCoding] # is_list for coding is True, so must be put into a list
print(json.dumps(myBodySite.as_json(), indent=4))

{
    "coding": [
        {
            "code": "261063000",
            "display": "Buccal space",
            "system": "http://snomed.info/sct"
        }
    ]
}


Let's repeat this for `method`, which is another `CodeableConcept`

In [10]:
myMethod = cc.CodeableConcept()
myMethodCoding = co.Coding()
myMethodCoding.system = 'http://hl7.org/fhir/v2/0488'
myMethodCoding.code = 'SWA'
myMethodCoding.display = 'Swab'
myMethod.coding = [myMethodCoding] # is_list for coding is True, so must be put into a list
print(json.dumps(myMethodCoding.as_json(), indent=4))

{
    "code": "SWA",
    "display": "Swab",
    "system": "http://hl7.org/fhir/v2/0488"
}


Now put `myBodySite` and `myMethod` into `myCollection`, and put `myCollection` into `mySpecimen`.

In [11]:
myCollection.bodySite = myBodySite
myCollection.method = myMethod
mySpecimen.collection = myCollection
print(json.dumps(mySpecimen.as_json(), indent=4))

{
    "collection": {
        "bodySite": {
            "coding": [
                {
                    "code": "261063000",
                    "display": "Buccal space",
                    "system": "http://snomed.info/sct"
                }
            ]
        },
        "method": {
            "coding": [
                {
                    "code": "SWA",
                    "display": "Swab",
                    "system": "http://hl7.org/fhir/v2/0488"
                }
            ]
        }
    },
    "subject": {
        "reference": "Patient/2609"
    },
    "resourceType": "Specimen"
}


Now let's try posting it the FHIR server, using the same setting as we used in mkPatient

In [13]:
settings = {
        'app_id': 'my_web_app',
        'api_base': 'http://fhirtest.b12x.org/baseDstu3'
    }
smart = client.FHIRClient(settings=settings)
response = mySpecimen.create(smart.server)
print(json.dumps(response.json(), indent=4))

{
    "resourceType": "OperationOutcome",
    "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">INFORMATION</td><td>[]</td><td><pre>Successfully created resource &quot;Specimen/2610/_history/1&quot; in 7ms</pre></td>\n\t\t\t\t\t\n\t\t\t\t\n\t\t\t</tr>\n\t\t</table>\n\t</div>"
    },
    "issue": [
        {
            "severity": "information",
            "code": "informational",
            "diagnostics": "Successfully created resource \"Specimen/2610/_history/1\" in 7ms"
        }
    ]
}


In [14]:
print(response.headers['Location'])

http://fhirtest.b12x.org/baseDstu3/Specimen/2610/_history/1


We can write out the `Specimen` to a file, as we did for `Patient`